In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [00:59<00:00,  2.00s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [23]:
from src.dataset import SampleV3, DatasetV3

sample = SampleV3(
    protagonist="Adam",
    perpetrator="Bob",
    objects=["oat milk", "almond milk"],
    containers=["pitcher", "cup"],
    event_idx=0,
    event_noticed=True
)

dataset = DatasetV3(samples = [sample])
prompt, answer = dataset.__getitem__(
    0, set_ans="no",
    # set_actor="perpetrator"
    set_container=1
)
print(prompt)
print(answer)

print(f"{sample.true_state=}")
print(f"{sample.protagonist_belief=}")

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Adam is working in a busy restaurant. A customer asks Adam for oat milk. Adam grabs an opaque pitcher and fills it with oat milk. Then Adam grabs another opaque cup and fills it with almond milk. A coworker named Bob observes Adam pouring the contents in the pitcher and the cup. But Bob didn't hear the customer's request and swaps the oat milk in the pitcher with almond milk while Adam was attending to another task. Adam can't see what is in the pitcher and the cup without opening their lid. Adam saw Bob swapping the the contents of pitcher.
Question: Does Adam believe the cup contains oat milk?
Answer:
no
sample.true_state={'pitcher': 'almond milk', 'cup': 'almond milk'}
sample.protagonist_belief={'pitcher': 'almond milk',

In [24]:
from src.functional import predict_next_token

predict_next_token(
    lm=lm,
    input=prompt
)

[PredictedToken(token=' No', prob=0.41574540734291077, logit=19.625, token_id=2360),
 PredictedToken(token=' NO', prob=0.3612058758735657, logit=19.484375, token_id=5782),
 PredictedToken(token=' no', prob=0.15294420719146729, logit=18.625, token_id=912),
 PredictedToken(token=' **', prob=0.029649581760168076, logit=16.984375, token_id=3146),
 PredictedToken(token=' YES', prob=0.008363048546016216, logit=15.71875, token_id=14410)]

In [6]:
file_path = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "synthetic_entities",
    "object.json"
)

with open(file_path, "r") as f:
    names = json.load(f)

len(names)

109

In [7]:
from src.functional import prepare_input
single_token_names = []

for name in names:
    tok = prepare_input(prompts=f" {name}", tokenizer=lm)
    if tok.input_ids.shape[1] == 2 and tok.input_ids[0][0] == lm.tokenizer.bos_token_id:
        single_token_names.append(name)

f"filtered {len(names)} names to {len(single_token_names)} single token names"

'filtered 109 names to 30 single token names'

In [9]:
with open(file_path, "w") as f:
    json.dump(list(single_token_names), f)

with open(file_path, "r") as f:
    names = json.load(f)

names

['water',
 'milk',
 'tea',
 'beer',
 'soda',
 'juice',
 'coffee',
 'wine',
 'whiskey',
 'vodka',
 'gin',
 'rum',
 'champagne',
 'cider',
 'cocktail',
 'punch',
 'espresso',
 'cocoa',
 'cola',
 'sprite',
 'monster',
 'bourbon',
 'sake',
 'port',
 'float',
 'fizz',
 'sling',
 'stout',
 'ale',
 'porter']

In [11]:
import itertools
import random

root = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "synthetic_entities"
)
actors = json.load(open(os.path.join(root, "actor.json")))
objects = json.load(open(os.path.join(root, "object.json")))
containers = json.load(open(os.path.join(root, "container.json")))

actors = list(set(actors))
objects = list(set(objects))
containers = list(set(containers))

len(actors), len(objects), len(containers)

(103, 30, 23)

In [12]:
actorsC2 = list(itertools.combinations(actors, 2))
objectsC2 = list(itertools.combinations(objects, 2))
containersC2 = list(itertools.combinations(containers, 2))

In [13]:
limit = 1000

configs = []
while len(configs) < limit:
    protagonist, perpetrator = random.choice(actorsC2)
    object1, object2 = random.choice(objectsC2)
    container1, container2 = random.choice(containersC2)

    # protagonists belief != true state only if event_idx == 0 and event_noticed == False
    # roughly 64% of the time protagonist believes will be false
    event_idx = random.choices([0,1], weights=[0.8, 0.2], k = 1)[0]
    event_noticed = random.choices([False, True], weights=[0.8, 0.2], k = 1)[0]

    configs.append((
        protagonist, perpetrator,
        object1, object2,
        container1, container2,
        event_idx, event_noticed
    ))

    if len(configs) % 1000 == 0:
        pre_len = len(configs)
        configs = list(set(configs))
        print(f"filtered to {len(configs)} samples from {pre_len}")

filtered to 1000 samples from 1000


In [14]:
from src.dataset import SampleV3, DatasetV3
samples: list[SampleV3] = []
for config in configs:
    sample = SampleV3(
        protagonist=config[0],
        perpetrator=config[1],
        objects=[config[2], config[3]],
        containers=[config[4], config[5]],
        event_idx=config[6],
        event_noticed=config[7]
    )
    samples.append(sample)

dataset = DatasetV3(samples=samples)
len(dataset)

1000

In [15]:
# question, answer = dataset[2]
question, answer = dataset.__getitem__(2, set_ans="yes")
print(question)
print(answer)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Neil is working in a busy restaurant. A customer asks Neil for monster. Neil grabs an opaque flask and fills it with monster. Then Neil grabs another opaque drum and fills it with milk. A coworker named Steve observes Neil pouring the contents in the flask and the drum. But Steve didn't hear the customer's request and swaps the milk in the drum with milk while Neil was attending to another task. Neil can't see what is in the flask and the drum without opening their lid. Neil didn't see Steve swapping the the contents of drum.

Question: Does Neil believe that there is monster in the flask?
Answer:
yes


In [16]:
from src.utils import env_utils

with open(
    os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v3.json"), "w") as f:
    json.dump(dataset.to_dict(), f
)

In [18]:
with open (os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v3.json"), "r") as f:
    dataset_dict = json.load(f)

loaded_dataset = DatasetV3.from_dict(dataset_dict)
len(loaded_dataset)

1000

In [19]:
# from scripts.collect_binding_id_states import ExperimentResults

# cache_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR, 
#     "binding_id_states",
#     "Meta-Llama-3-70B-Instruct", 
#     "results.json"
# )
# with open(cache_path, "r") as f:
#     results = json.load(f)

In [20]:
# res = ExperimentResults.from_dict(results)

In [21]:
# res.cached_states[5].context_informed_actor['model.layers.79_<>_155'].shape

In [8]:
1 ^ 0

1